# Using Neptune with plotting libraries

<a target="_blank" href="https://colab.research.google.com/github/neptune-ai/examples/blob/main/integrations-and-supported-tools/plotting-libraries/notebooks/Neptune_Plotting_Support.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/>
</a><a target="_blank" href="https://github.com/neptune-ai/examples/blob/main/integrations-and-supported-tools/plotting-libraries/scripts/Neptune_Plotting_Support.py">
  <img alt="Open in GitHub" src="https://img.shields.io/badge/Open_in_GitHub-blue?logo=github&labelColor=black">
</a><a target="_blank" href="https://app.neptune.ai/o/showcase/org/plotting/runs/details?viewId=standard-view&detailsTab=metadata&shortId=PLOT-2&type=run&path=&attribute=altair"> 
  <img alt="Explore in Neptune" src="https://neptune.ai/wp-content/uploads/2024/01/neptune-badge.svg">
</a><a target="_blank" href="https://docs.neptune.ai/logging/interactive_visualizations/">
  <img alt="View tutorial in docs" src="https://neptune.ai/wp-content/uploads/2024/01/docs-badge-2.svg">
</a>

## Introduction

This guide will show you how to:

* Initialize Neptune and create a run,
* Log plots created using the following libraries:
    * [Altair](Neptune_Plotting_Support.ipynb#log-altair-charts-to-neptune)
    * [Bokeh](Neptune_Plotting_Support.ipynb#log-bokeh-charts-to-neptune)
    * [Folium (Leaflet)](Neptune_Plotting_Support.ipynb#log-folium-leaflet-maps-to-neptune)
    * [Matplotlib](Neptune_Plotting_Support.ipynb#log-matplotlib-charts-to-neptune)
    * [Plotly](Neptune_Plotting_Support.ipynb#log-plotly-charts-to-neptune)
    * [Seaborn](Neptune_Plotting_Support.ipynb#log-seaborn-charts-to-neptune)

## Before you start

This notebook example lets you try out Neptune as an anonymous user, with zero setup.

If you want to see the example logged to your own workspace instead:

  1. Create a Neptune account. [Register &rarr;](https://neptune.ai/register)
  1. Create a Neptune project that you will use for tracking metadata. For instructions, see [Creating a project](https://docs.neptune.ai/setup/creating_project) in the Neptune docs.

### Install Neptune

In [ ]:
%pip install -U -q neptune

### Import and initialize Neptune

In [ ]:
import neptune

To create a new run for tracking the metadata, you tell Neptune who you are (`api_token`) and where to send the data (`project`).

You can use the default code cell below to create an anonymous run in a public project. **Note**: Public projects are cleaned regularly, so anonymous runs are only stored temporarily.

### Log to your own project instead

Replace the code below with the following:

```python
from getpass import getpass

run = neptune.init_run(
    project="workspace-name/project-name",  # replace with your own (see instructions below)
    api_token=getpass("Enter your Neptune API token: "),
)
```

To find your API token and full project name:

1. [Log in to Neptune](https://app.neptune.ai/).
1. In the bottom-left corner, expand your user menu and select **Get your API token**.
1. The workspace name is displayed in the top-left corner of the app. 

    To copy the project path, in the top-right corner, open the settings menu and select **Properties**.

For more help, see [Setting Neptune credentials](https://docs.neptune.ai/setup/setting_credentials) in the Neptune docs.

In [ ]:
run = neptune.init_run(
    api_token=neptune.ANONYMOUS_API_TOKEN,
    project="common/plotting",
    tags=["notebook"],
)

**To open the run in the Neptune web app, click the link that appeared in the cell output.**

We'll use the `run` object we just created to log metadata. You'll see the metadata appear in the app.

## Log Altair charts to Neptune

### Create a sample chart

In [ ]:
%pip install -U -q altair vega_datasets

In [ ]:
import altair as alt
from vega_datasets import data

source = data.cars()

brush = alt.selection_interval()

points = (
    alt.Chart(source)
    .mark_point()
    .encode(
        x="Horsepower:Q",
        y="Miles_per_Gallon:Q",
        color=alt.condition(brush, "Origin:N", alt.value("lightgray")),
    )
    .add_params(brush)
)

bars = (
    alt.Chart(source)
    .mark_bar()
    .encode(y="Origin:N", color="Origin:N", x="count(Origin):Q")
    .transform_filter(brush)
)

chart = points & bars

chart

### Log interactive chart to Neptune

In [ ]:
run["altair"].upload(chart)

## Log Bokeh charts to Neptune

### Create a sample chart

In [ ]:
%pip install -U -q bokeh

In [ ]:
import numpy as np
from bokeh.plotting import figure, show, output_notebook

N = 500
x = np.linspace(0, 10, N)
y = np.linspace(0, 10, N)
xx, yy = np.meshgrid(x, y)
d = np.sin(xx) * np.cos(yy)

p = figure(tooltips=[("x", "$x"), ("y", "$y"), ("value", "@image")])
p.x_range.range_padding = p.y_range.range_padding = 0

# must give a vector of image data for image parameter
p.image(image=[d], x=0, y=0, dw=10, dh=10, palette="Spectral11", level="image")
p.grid.grid_line_width = 0.5

# Display plot
output_notebook()
show(p)

### Log interactive chart to Neptune

In [ ]:
run["bokeh"].upload(p)

## Log folium (leaflet) maps to Neptune

### Create a sample map

In [ ]:
%pip install -U -q folium

In [ ]:
import folium

m = folium.Map()
m

### Log interactive map to Neptune

In [ ]:
# Save map as an HTML file
m.save("map.html")

# Upload HTML file to Neptune
run["folium"].upload("map.html")

## Log matplotlib charts to Neptune

### Create a sample chart

In [ ]:
%pip install -U -q matplotlib

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

np.random.seed(42)
data = np.random.randn(2, 100)

figure, ax = plt.subplots(2, 2, figsize=(5, 5))
ax[0, 0].hist(data[0])
ax[1, 0].scatter(data[0], data[1])
ax[0, 1].plot(data[0], data[1])

figure.show()

### Log static chart to Neptune

In [ ]:
run["matplotlib-static"].upload(figure)

### Log interactive chart to Neptune

In [ ]:
%pip install -U -q plotly

In [ ]:
from neptune.types import File

run["matplotlib-interactive"].upload(File.as_html(figure))

## Log Plotly charts to Neptune

### Create a sample chart

In [ ]:
import plotly.express as px

df = px.data.iris()
fig = px.scatter_3d(df, x="sepal_length", y="sepal_width", z="petal_width", color="species")

# fig.show() # Uncomment to preview the chart

### Log interactive chart to Neptune

In [ ]:
run["plotly"].upload(fig)

## Log Seaborn charts to Neptune

### Create a sample chart

In [ ]:
%pip install -U -q seaborn

In [ ]:
import seaborn as sns

df = sns.load_dataset("penguins")
plot = sns.pairplot(df, hue="species")
plot;

### Log chart to Neptune

In [ ]:
run["seaborn"].upload(plot)

## Stop logging

In [ ]:
run.stop()

## Explore the charts in Neptune

The charts can be found in the **All metadata** section.

You can also explore this example run: https://app.neptune.ai/o/showcase/org/plotting/runs/details?viewId=standard-view&detailsTab=metadata&shortId=PLOT-2&type=run&path=&attribute=altair

## What's next

Now that you know how to log charts to Neptune, you can learn:

* [How to log other types of metadata to Neptune](https://docs.neptune.ai/logging/what_you_can_log/)
* [How to download runs data from Neptune](https://docs.neptune.ai/usage/querying_metadata/)
* [How to integrate Neptune with the ML framework you are using](https://docs.neptune.ai/integrations/)